Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Crawl and save in dataframe

In [4]:
#tên, năm ra mắt, thể loại, đạo diễn, thời lượng, score, trạng thái, ngôn ngữ, ngân sách, doanh thu
name = []
release_year = []
genre = []
director = []
runtime = []
score = []
status = []
language = []
budget = []
revenue = []

#lấy nội dung html
for page in range(250):
    root_url = 'https://www.themoviedb.org/movie/'
    base_url = root_url + '?page=' + str(page + 1)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        content = response.content.decode('utf-8', errors='replace')
        soup = BeautifulSoup(content, 'html.parser')
    else:
        print('Failed to retrieve web data from the website.')

    #lấy đường dẫn của các movie cụ thể
    set_movie_id = set()
    for a_tag in soup.find_all('a', href=re.compile(r'/movie/')):
        match = re.search(r'/movie/(\d+)', a_tag['href'])
        if match:
            set_movie_id.add(match.group(1))
    list_movie_id = list(set_movie_id)
    
    #truy cập vào các movie cụ thể để lấy data
    for movie_id in list_movie_id:
        movie_url = root_url + movie_id
        response = requests.get(movie_url, headers=headers)
        if response.status_code == 200:
            content = response.content.decode('utf-8', errors='replace')
            soup = BeautifulSoup(content, 'html.parser')
        else:
            print('Failed to retrieve movie data from the website.')
            break
        #name
        name_ = soup.find('div', class_='title').find('a').text
        name.append(name_)

        #release year
        year_ = soup.find('span', class_='tag release_date').text.strip("()") 
        release_year.append(year_)
        
        #genres
        genres = [a.text for a in soup.find('span', class_='genres').find_all('a')]
        genre.append(genres)

        #runtime
        runtime_ = soup.find('span', class_='runtime').text.strip() if soup.find('span', class_='runtime') is not None else ' -'
        runtime.append(runtime_)

        #score
        score_ = soup.find('div', class_='user_score_chart')['data-percent']
        score.append(score_)

        #director
        director_ = soup.find('li', class_='profile').find('a').text if soup.find('li', class_='profile') is not None else ' -'
        director.append(director_)

        #status, language, budget, revenue
        p_section_element = soup.find('section', class_='facts left_column').find_all('p') 
        for ele in p_section_element:
            text = ele.get_text(strip=True)
            if text.startswith('Status'):
                status_ = text.split('Status')[1]
            elif text.startswith('Original Language'):
                language_ = text.split('Original Language')[1]
            elif text.startswith('Budget'):
                budget_ = text.split('Budget')[1]
            elif text.startswith('Revenue'):
                 revenue_ = text.split('Revenue')[1]
        status.append(status_)
        language.append(language_)
        budget.append(budget_)
        revenue.append(revenue_)
    print('Page ' + str(page + 1) + ' done!')

Page 1 done!
Page 2 done!
Page 3 done!
Page 4 done!
Page 5 done!
Page 6 done!
Page 7 done!
Page 8 done!
Page 9 done!
Page 10 done!
Page 11 done!
Page 12 done!
Page 13 done!
Page 14 done!
Page 15 done!
Page 16 done!
Page 17 done!
Page 18 done!
Page 19 done!
Page 20 done!
Page 21 done!
Page 22 done!
Page 23 done!
Page 24 done!
Page 25 done!
Page 26 done!
Page 27 done!
Page 28 done!
Page 29 done!
Page 30 done!
Page 31 done!
Page 32 done!
Page 33 done!
Page 34 done!
Page 35 done!
Page 36 done!
Page 37 done!
Page 38 done!
Page 39 done!
Page 40 done!
Page 41 done!
Page 42 done!
Page 43 done!
Page 44 done!
Page 45 done!
Page 46 done!
Page 47 done!
Page 48 done!
Page 49 done!
Page 50 done!
Page 51 done!
Page 52 done!
Page 53 done!
Page 54 done!
Page 55 done!
Page 56 done!
Page 57 done!
Page 58 done!
Page 59 done!
Page 60 done!
Page 61 done!
Page 62 done!
Page 63 done!
Page 64 done!
Page 65 done!
Page 66 done!
Page 67 done!
Page 68 done!
Page 69 done!
Page 70 done!
Page 71 done!
Page 72 done!
P

In [5]:
movie_df = pd.DataFrame({'Name': name, 'Released year': release_year, 'Genre': genre, 'Director': director, 'Runtime': runtime,
                         'Score': score, 'Status': status, 'Language': language, 'Budget': budget, 'Revenue': revenue})
movie_df

,Name,Released year,Genre,Director,Runtime,Score,Status,Language,Budget,Revenue
0,Fast X,2023,"[Action, Crime, Thriller]",Dan Mazeau,2h 22m,72.0,Released,English,"$340,000,000.00","$704,709,660.00"
1,Trolls Band Together,2023,"[Animation, Family, Music, Fantasy, Comedy]",Thomas Dam,1h 32m,72.0,Released,English,"$95,000,000.00","$173,800,000.00"
2,Robot Apocalypse,2021,"[Science Fiction, Action]",Marcus Friedlander,1h 27m,21.0,Released,English,-,-
3,Five Nights at Freddy's,2023,"[Horror, Mystery]",Emma Tammi,1h 50m,78.0,Released,English,"$20,000,000.00","$286,700,000.00"
4,Oppenheimer,2023,"[Drama, History]",Christopher Nolan,3h 1m,81.0,Released,English,"$100,000,000.00","$951,000,000.00"
...,...,...,...,...,...,...,...,...,...,...
4995,Secret in Their Eyes,2015,"[Thriller, Mystery, Drama, Crime]",Billy Ray,1h 51m,64.0,Released,English,"$19,500,000.00","$34,854,990.00"
4996,The Vatican Tapes,2015,"[Thriller, Horror]",Mark Neveldine,1h 31m,53.0,Released,English,"$13,000,000.00","$1,784,763.00"
4997,Song to Song,2017,"[Romance, Drama, Music]",Terrence Malick,2h 9m,55.0,Released,English,"$10,000,000.00","$1,710,528.00"
4998,Divine Intervention,2023,[Comedy],Pedro Pablo Ibarra,1h 40m,82.0,Released,Spanish; Castilian,-,-


In [6]:
movie_df.to_csv('movie.csv', index=False)